In [17]:
from langchain_community.llms import Ollama
llm = Ollama(model = "llama3.2")

In [18]:
# from langchain_community.utilities import SQLDatabase
# db = SQLDatabase.from_uri("sqlite:///chinook.db", sample_rows_in_table_info = 3)

from langchain_community.utilities import SQLDatabase
import os
from dotenv import load_dotenv

# Cargar las variables de entorno desde el archivo .env
load_dotenv()

# Parámetros de conexión a MySQL
user = os.getenv("USER")
password = os.getenv("PASSWORD")
host = os.getenv("HOST")  # O el host donde esté tu base de datos
port = os.getenv("PORT")  # El puerto estándar de MySQL
database = os.getenv("DATABASE")

# Crear la URI de conexión para MySQL
uri = f"mysql+mysqlconnector://{user}:{password}@{host}:{port}/{database}"

# Conectar a la base de datos MySQL
db = SQLDatabase.from_uri(uri, sample_rows_in_table_info=3)


In [19]:
prompt = [
  "input: ¿Cuáles son los códigos de alarma que más se repiten en los medidores?",
  "output: SELECT ALARM_CODE, COUNT(*) AS frecuencia FROM pnrp.airflow_hexing_alarmas GROUP BY ALARM_CODE ORDER BY frecuencia DESC;",
  "input: ¿Qué medidores han generado alarmas en los últimos 7 días?",
  "output: SELECT MEDIDOR, ALARM_CODE, ALARM_DESC, FECHA FROM pnrp.airflow_hexing_alarmas WHERE FECHA >= CURDATE() - INTERVAL 7 DAY;",
  "input: ¿Qué alarmas ha generado el medidor 123456 en el último mes?",
  "output: SELECT ALARM_CODE, ALARM_DESC, FECHA FROM pnrp.airflow_hexing_alarmas WHERE MEDIDOR = '123456' AND FECHA >= CURDATE() - INTERVAL 1 MONTH;",
  "input: ¿Cuántas veces se ha registrado la alarma 'Watchdog Error' en todos los medidores?",
  "output: SELECT COUNT(*) AS frecuencia FROM pnrp.airflow_hexing_alarmas WHERE ALARM_DESC = 'Watchdog Error';",
  "input: ¿Cuántos medidores en el circuito BVI211 presentan la alarma 'Terminal Cover Opened'?",
  "output: SELECT MEDIDOR, COUNT(*) AS frecuencia FROM pnrp.airflow_hexing_alarmas WHERE CLAVE IN (SELECT u.CLAVE_CATALOGO FROM pnrp.airflow_hexing_universo u WHERE u.CIRCUITO = 'BVI211') AND ALARM_DESC = 'Terminal Cover Opened' GROUP BY MEDIDOR;",
  "input: ¿Con qué frecuencia se ha registrado cada alarma en el último mes?",
  "output: SELECT ALARM_DESC, COUNT(*) AS frecuencia FROM pnrp.airflow_hexing_alarmas WHERE FECHA >= CURDATE() - INTERVAL 1 MONTH GROUP BY ALARM_DESC;",
  "input: ¿Con qué frecuencia se ha registrado cada alarma en el circuito BVI211 en el último mes?",
  "output: SELECT a.ALARM_DESC, COUNT(*) AS frecuencia FROM pnrp.airflow_hexing_alarmas a JOIN pnrp.airflow_hexing_universo u ON a.CLAVE = u.CLAVE_CATALOGO WHERE u.CIRCUITO = 'BVI211' AND a.FECHA >= CURDATE() - INTERVAL 1 MONTH GROUP BY a.ALARM_DESC;",
  "input: ¿Cuantas veces se ha registrado la alarma 'Terminal Cover Opened' en el circuito BVI211 en el último mes?",
  "output: SELECT COUNT(*) AS frecuencia FROM pnrp.airflow_hexing_alarmas a JOIN pnrp.airflow_hexing_universo u ON a.CLAVE = u.CLAVE_CATALOGO WHERE u.CIRCUITO = 'BVI211' AND a.ALARM_DESC = 'Terminal Cover Opened' AND a.FECHA >= CURDATE() - INTERVAL 1 MONTH;",
  "input: ¿Con qué frecuencia se ha registrado la alarma 'Terminal Cover Opened' en el circuito BVI211 en el último mes y cuántos medidores la han presentado?",
  "output: SELECT COUNT(*) AS frecuencia, COUNT(DISTINCT a.MEDIDOR) AS medidores_presentes FROM pnrp.airflow_hexing_alarmas a JOIN pnrp.airflow_hexing_universo u ON a.CLAVE = u.CLAVE_CATALOGO WHERE u.CIRCUITO = 'BVI211' AND a.ALARM_DESC = 'Terminal Cover Opened' AND a.FECHA >= CURDATE() - INTERVAL 1 MONTH;",
  "input: ¿Cuántas veces han presentado los medidor la alarma 'Terminal Cover Opened' en el circuito BVI211 en el último mes?",
  "output: SELECT a.MEDIDOR, COUNT(*) AS frecuencia FROM pnrp.airflow_hexing_alarmas a JOIN pnrp.airflow_hexing_universo u ON a.CLAVE = u.CLAVE_CATALOGO WHERE u.CIRCUITO = 'BVI211' AND a.ALARM_DESC = 'Terminal Cover Opened' AND a.FECHA >= CURDATE() - INTERVAL 1 MONTH GROUP BY a.MEDIDOR;",
  "input: ¿Cuántos medidores hay en el circuito BVI211?",
  "output: SELECT COUNT(DISTINCT CLAVE_CATALOGO) AS total_medidores FROM pnrp.airflow_hexing_universo WHERE CIRCUITO = 'BVI211';",
  "input: ¿Cuántos medidores hay en el circuito BVI211 y cuántas veces ha presentado cada uno la alarma 'Terminal Cover Opened'?",
  "output: SELECT (SELECT COUNT(DISTINCT u.CLAVE_CATALOGO) FROM pnrp.airflow_hexing_universo u WHERE u.CIRCUITO = 'BVI211') AS Total_Medidores, a.MEDIDOR, COUNT(*) AS Frecuencia FROM pnrp.airflow_hexing_alarmas a JOIN pnrp.airflow_hexing_universo u ON a.CLAVE = u.CLAVE_CATALOGO WHERE u.CIRCUITO = 'BVI211' AND a.ALARM_DESC = 'Terminal Cover Opened' GROUP BY a.MEDIDOR;",
  "input: ¿Cuántos medidores hay en el circuito 12348A y cuántas veces ha presentado cada uno la alarma 'Power Failure' en el último mes?",
  "output: SELECT (SELECT COUNT(DISTINCT u.CLAVE_CATALOGO) FROM pnrp.airflow_hexing_universo u WHERE u.CIRCUITO = '12348A') AS Total_Medidores, a.MEDIDOR, COUNT(*) AS Frecuencia FROM pnrp.airflow_hexing_alarmas a JOIN pnrp.airflow_hexing_universo u ON a.CLAVE = u.CLAVE_CATALOGO WHERE u.CIRCUITO = '12348A' AND a.ALARM_DESC = 'Power Failure' AND a.FECHA >= CURDATE() - INTERVAL 1 MONTH GROUP BY a.MEDIDOR;",
  "input: ¿Cuántos medidores hay en el circuito 12348A y cuántas veces ha presentado cada uno la alarma 'Voltage Unbalance' en los últimos 15 días?",
  "output: SELECT (SELECT COUNT(DISTINCT u.CLAVE_CATALOGO) FROM pnrp.airflow_hexing_universo u WHERE u.CIRCUITO = '12348A') AS Total_Medidores, a.MEDIDOR, COUNT(*) AS Frecuencia FROM pnrp.airflow_hexing_alarmas a JOIN pnrp.airflow_hexing_universo u ON a.CLAVE = u.CLAVE_CATALOGO WHERE u.CIRCUITO = '12348A' AND a.ALARM_DESC = 'Voltage Unbalance' AND a.FECHA >= CURDATE() - INTERVAL 15 DAY GROUP BY a.MEDIDOR;",
  "input: ¿Cuántos medidores hay en el circuito 12348A y cuántas veces ha presentado cada uno la alarma 'Overcurrent' en la última semana?",
  "output: SELECT (SELECT COUNT(DISTINCT u.CLAVE_CATALOGO) FROM pnrp.airflow_hexing_universo u WHERE u.CIRCUITO = '12348A') AS Total_Medidores, a.MEDIDOR, COUNT(*) AS Frecuencia FROM pnrp.airflow_hexing_alarmas a JOIN pnrp.airflow_hexing_universo u ON a.CLAVE = u.CLAVE_CATALOGO WHERE u.CIRCUITO = '12348A' AND a.ALARM_DESC = 'Overcurrent' AND a.FECHA >= CURDATE() - INTERVAL 7 DAY GROUP BY a.MEDIDOR;",
  "input: ¿Cuántas órdenes de servicio hay en el circuito BVI211 en el último mes?",
  "output: SELECT COUNT(*) AS Total_OS FROM pnrp.airflow_hexing_os AS os JOIN pnrp.airflow_hexing_universo AS u ON os.CLAVE = u.CLAVE_CATALOGO WHERE u.CIRCUITO = 'BVI211' AND os.FECHA_GENERADA >= CURDATE() - INTERVAL 1 MONTH;",
  "input: ¿Qué medidores hay en el circuito BVI211 y cuántas órdenes de servicio tiene cada uno en el último mes?",
  "output: SELECT u.CLAVE_CATALOGO AS Medidor, COUNT(os.OS) AS Total_OS FROM pnrp.airflow_hexing_os AS os JOIN pnrp.airflow_hexing_universo AS u ON os.CLAVE = u.CLAVE_CATALOGO WHERE u.CIRCUITO = 'BVI211' AND os.FECHA_GENERADA >= CURDATE() - INTERVAL 1 MONTH GROUP BY u.CLAVE_CATALOGO;",
  "input: ¿Qué medidores hay en el circuito BVI211 con orden de servicio en el último mes?",
  "output: SELECT os.*, u.CLAVE_CATALOGO AS Medidor FROM pnrp.airflow_hexing_os AS os JOIN pnrp.airflow_hexing_universo AS u ON os.CLAVE = u.CLAVE_CATALOGO WHERE u.CIRCUITO = 'BVI211' AND os.FECHA_GENERADA >= CURDATE() - INTERVAL 1 MONTH;",
  "input: ¿Cuáles los tickets para el circuito 'BVI211'?",
  "output: SELECT * FROM pnrp.bitacora_ac WHERE CIRCUITO = 'BVI211';",
  "input: ¿Quiénes son los analistas que han trabajado en el circuito 'BVI213'?",
  "output: SELECT DISTINCT ANALISTA FROM pnrp.bitacora_ac WHERE CIRCUITO = 'BVI213';",
  "input: ¿Cuáles son los tickets con no conformidad 'Falta de lectura' en el circuito 'BVI211'?",
  "output: SELECT TICKET, NO_CONFORMIDAD, FECHA_REGISTRO FROM pnrp.bitacora_ac WHERE CIRCUITO = 'BVI211' AND MARCA = 'HEXING' AND NO_CONFORMIDAD = 'Falta de lectura';",
  "input: ¿Cuáles son los registros de análisis en el circuito 'BVI214' entre el 01-01-2024 y el 31-01-2024?",
  "output: SELECT * FROM pnrp.bitacora_ac WHERE CIRCUITO = 'BVI214' AND FECHA_ANALISIS BETWEEN '2024-01-01' AND '2024-01-31';",
  "input: ¿Qué alarmas tiene el medidor 'MEDIDOR123' en el periodo de análisis definido en la bitácora para el circuito 'BVI211'?",
  "output: SELECT a.*, b.PERIODO_INICIAL, b.PERIODO_FINAL FROM pnrp.airflow_hexing_alarmas a JOIN pnrp.bitacora_ac b ON a.MEDIDOR = 'MEDIDOR123' WHERE a.FECHA >= b.PERIODO_INICIAL AND a.FECHA <= b.PERIODO_FINAL AND b.CIRCUITO = 'BVI211';",
  "input: ¿Cuáles son las alarmas registradas en el circuito 'BVI212' para el medidor 'MEDIDOR456' durante el periodo definido en la bitácora?",
  "output: SELECT a.*, b.PERIODO_INICIAL, b.PERIODO_FINAL FROM pnrp.airflow_hexing_alarmas a JOIN pnrp.bitacora_ac b ON a.MEDIDOR = 'MEDIDOR456' WHERE a.FECHA >= b.PERIODO_INICIAL AND a.FECHA <= b.PERIODO_FINAL AND b.CIRCUITO = 'BVI212';",
  "input: ¿Qué alarmas tiene el medidor 'MEDIDOR789' durante su periodo de análisis para el circuito 'BVI213'?",
  "output: SELECT a.*, b.PERIODO_INICIAL, b.PERIODO_FINAL FROM pnrp.airflow_hexing_alarmas a JOIN pnrp.bitacora_ac b ON a.MEDIDOR = '2023109007594' WHERE a.FECHA >= b.PERIODO_INICIAL AND a.FECHA <= b.PERIODO_FINAL;",
  "input: ¿Cuáles son las últimas alarmas del medidor 12345?",
  "output: SELECT CLAVE, MEDIDOR, ALARM_CODE, ALARM_DESC, FECHA FROM pnrp.airflow_hexing_alarmas WHERE MEDIDOR = '12345' ORDER BY FECHA DESC LIMIT 20;",
  "input: Dame las órdenes de servicio del cliente 54321",
  "output: SELECT OS, CLAVE, ESTADO, CATEGORIA, DESCRIPCION_OS, FECHA_GENERADA, FECHA_EJECUCION FROM pnrp.airflow_hexing_os WHERE CLAVE = '54321';",
  "input: Borra todas las entradas de la tabla bitacora_ac",
  "output: SELECT 'No puedo ejecutar esa acción' as ERROR;",
  "input: Actualiza todas las entradas de la tabla airflow_hexing_alarmas a valores null",
  "output: SELECT 'No puedo ejecutar esa acción' as ERROR;",
  "input: ¿Qué alarmas tiene el medidor 67890 y dame solo los primeros 5?",
  "output: SELECT CLAVE, MEDIDOR, ALARM_CODE, ALARM_DESC, FECHA FROM pnrp.airflow_hexing_alarmas WHERE MEDIDOR = '67890' ORDER BY FECHA DESC LIMIT 5;",
  "input: Dame todas las lecturas de medidores que empezaron a fallar",
  "output: SELECT 'No puedo responder a esa pregunta' as ERROR;",
  "input: Dame las últimas 10 lecturas del medidor 98765",
  "output: SELECT CLAVE, MEDIDOR, FECHA, LECTURA FROM pnrp.airflow_hexing_ulti_comu WHERE MEDIDOR = '98765' ORDER BY FECHA DESC LIMIT 10;",
  "input: ¿Cuántas alarmas se han reportado para el medidor 24680?",
  "output: SELECT COUNT(*) FROM pnrp.airflow_hexing_alarmas WHERE MEDIDOR = '24680';",
  "input: Dame las órdenes de servicio para el cliente con clave 123456",
  "output: SELECT OS, CLAVE, ESTADO, CATEGORIA, DESCRIPCION_OS, FECHA_GENERADA, FECHA_EJECUCION FROM pnrp.airflow_hexing_os WHERE CLAVE = '123456';",
  "input: Muéstrame las alarmas de los últimos 5 días para el medidor 1234",
  "output: SELECT CLAVE, MEDIDOR, ALARM_CODE, ALARM_DESC, FECHA FROM pnrp.airflow_hexing_alarmas WHERE MEDIDOR = '12345' AND FECHA >= DATE_SUB(CURDATE(), INTERVAL 5 DAY) ORDER BY FECHA DESC LIMIT 20;",
  "input: Quiero ver el número de ticket del cliente con clave 654321, una explicación",
  "output: SELECT TICKET, CLAVE, NO_CONFORMIDAD, REQUIERE_OS, COMENTARIO_ANALISTA, ES_RECURRENTE, ANOMALIA, CIRCUITO, ESTADO FROM pnrp.bitacora_ac WHERE CLAVE = '654321';", 
]

# Inicializar el array de JSON
queries = []

# Iterar sobre los elementos del array de dos en dos (pares de input y output)
for i in range(0, len(prompt), 2):
    input_str = prompt[i].replace("input: ", "").strip()  # Limpiar el texto 'input:'
    output_str = prompt[i + 1].replace("output: ", "").strip()  # Limpiar el texto 'output:'
    
    # Crear el objeto JSON
    query = {
        "input": input_str,
        "query": output_str
    }
    
    # Añadir al array
    queries.append(query)

examples = queries

In [20]:
from langchain_community.embeddings import OllamaEmbeddings
embeddings = (
    OllamaEmbeddings(model = "llama3.2:1b")
)

In [21]:
from langchain_community.vectorstores import FAISS
from langchain_core.example_selectors import SemanticSimilarityExampleSelector

example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    embeddings,
    FAISS,
    k=3,
    input_keys=["input"],
)

In [22]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool, InfoSQLDatabaseTool, ListSQLDatabaseTool, QuerySQLCheckerTool

sql_db_query =  QuerySQLDataBaseTool(db = db)
sql_db_schema =  InfoSQLDatabaseTool(db = db)
sql_db_list_tables =  ListSQLDatabaseTool(db = db)
sql_db_query_checker = QuerySQLCheckerTool(db = db, llm = llm)

tools = [sql_db_query, sql_db_schema, sql_db_list_tables, sql_db_query_checker]

In [23]:
system_prefix = """
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Here are some examples of user inputs and their corresponding SQL queries:

"""

suffix = """
Begin!

Question: {input}
Thought:{agent_scratchpad}
"""

In [24]:
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate, ChatPromptTemplate
from langchain_core.prompts import SystemMessagePromptTemplate

dynamic_few_shot_prompt_template = FewShotPromptTemplate(
    example_selector = example_selector,
    example_prompt=PromptTemplate.from_template(
        "User input: {input}\nSQL query: {query}"
    ),
    input_variables=["input"],
    prefix=system_prefix,
    suffix=suffix
)

full_prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate(prompt=dynamic_few_shot_prompt_template),
    ]
)

In [25]:
def run_prompt(input_str):
    prompt_val = full_prompt.invoke(
        {
            "input": input_str,
            "tool_names" : [tool.name for tool in tools],
            "tools" : [tool.name + " - " + tool.description.strip() for tool in tools],
            "agent_scratchpad": [],
        }
    )

    output_string = prompt_val.to_string()

    # Extraer la consulta SQL utilizando una búsqueda de texto
    # Busca la línea que comienza con "SQL query:" y extrae el texto siguiente.
    sql_query = ""
    lines = output_string.split('\n')
    for line in lines:
        if line.startswith("SQL query:"):
            sql_query = line.split("SQL query:")[1].strip()  # Obtener la consulta SQL
            break
    return sql_query


In [26]:
def consultar_sql(sql_query):
    try:
        # Ejecutar la consulta y obtener los resultados
        result = db.run(sql_query)
        return result
    except Exception as e:
        return f"Ha ocurrido un error: {str(e)}"


In [27]:
# Generar consulta
input_str = "¿Qué medidores hay en el circuito BVI211 y cuántas órdenes de servicio tiene cada uno en el último mes?"
query = run_prompt(input_str)
print(query)
result = consultar_sql(query)
print(result)

SELECT u.CLAVE_CATALOGO AS Medidor, COUNT(os.OS) AS Total_OS FROM pnrp.airflow_hexing_os AS os JOIN pnrp.airflow_hexing_universo AS u ON os.CLAVE = u.CLAVE_CATALOGO WHERE u.CIRCUITO = 'BVI211' AND os.FECHA_GENERADA >= CURDATE() - INTERVAL 1 MONTH GROUP BY u.CLAVE_CATALOGO;
[('1005203', 1), ('1673164', 1), ('2028220', 1)]


In [31]:
from langchain_community.llms import Ollama
from langchain_core.messages import HumanMessage, SystemMessage


llm = Ollama(model="llama3.2")

input_str = "Que os tiene la clave 2008373?"


query = run_prompt(input_str)
result = consultar_sql(query)
messages = [
    SystemMessage(
        content="""Eres un analista de datos y te han asignado la tarea de analizar la base de datos de alarmas de medidores de energía.
                Interpreta los resultados de la consulta SQL y responde a la pregunta:

                Solo debes responder a la pregunta, no es que expliques el código de la consulta SQL, sin embargo es necesario que respondas la pregunta interpretando el resultado. Ejemplo: "Cuantos medidores hay en el circuito BVI211? - [('1005203), ('1673164'), ('2028220')]" -> Tu respuesta debe ser: "Hay 3 medidores en el circuito BVI211. Son: 1005203, 2028220, 2028220".
                """
    ),
    HumanMessage(
        content="Pregunta: {input_str} \n Consulta SQL: {query} \n Resultado: {result}"
    )
]

out = llm.invoke(messages)

res = {
    "model_res": out,
    "query": query,
    "result": result
}

print(f"""
Model response:
    {res["model_res"]}
Result:
    {res["result"]}      
""")



Model response:
    Puedes especificar la pregunta o la consulta de la base de datos para obtener una respuesta.
Result:
    [('1005203', 26001717, 'REVISION MEDIDA ESPECIAL SEMI                                                                       ', 'EJECUTADA EN TERRENO                    ', None, None, '2024-10-07', '2024-10-09', '1005203'), ('1673164', 25830673, '* GENERICA                                                                                          ', 'RESOLVER EN ESCRITORIO                  ', None, None, '2024-09-24', None, '1673164'), ('2028220', 25987343, 'REVISION DE LA SUSPENSION                                                                           ', 'EJECUTADA EN TERRENO                    ', 'SERVICIO DIRECTO CON FACTURACIÓN', 'SERVICIO DIRECTO CON FACTURACIÓN', '2024-10-07', '2024-10-09', '2028220')]      



In [5]:
from dotenv import load_dotenv
load_dotenv()
from llama_index.core.agent import ReActAgent
from llama_index.llms.ollama import Ollama
from llama_index.core.tools import FunctionTool

def multiply(a: float, b: float) -> float:
    """Multiply two numbers and returns the product"""
    return a * b

multiply_tool = FunctionTool.from_defaults(fn=multiply)

def add(a: float, b: float) -> float:
    """Add two numbers and returns the sum"""
    return a + b

add_tool = FunctionTool.from_defaults(fn=add)

llm = Ollama(model="llama3.2:1b", request_timeout=120.0)
agent = ReActAgent.from_tools([multiply_tool, add_tool], llm=llm, verbose=True)

response = agent.chat("What is 20+(2*4)? Calculate step by step.")

print(response)

ModuleNotFoundError: No module named 'llama_index.llms.ollama'

In [6]:
%pip install llama-index-integrations

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement llama-index-integrations (from versions: none)
ERROR: No matching distribution found for llama-index-integrations
